## Query header - Gen3

In this notebook, we show several ways to query the fits headers\
Craig Lage - 16-Mar-21

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pandas as pd
import lsst.daf.butler as daf_butler
from lsst.obs.lsst import LsstCam, LsstTS8
import lsst.eo.pipe as eo_pipe

In [ ]:
repo = '/repo/ir2'
butler = daf_butler.Butler(repo, collections=["LSSTCam/raw/all","LSSTCam/calib"])
 
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.science_program='13391'"):
    exposureList.append([record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name])
exposureList = sorted(exposureList, key=lambda x: x[0])   
for item in exposureList:
    #print(f"{item[0]} \t {item[1]} \t {item[2]} \t\t {item[3]} \t {item[4]}")
    print(item)

In [ ]:
repo = '/repo/ir2'
butler = daf_butler.Butler(repo)

In [ ]:
butler.registry.queryCollections(...)

In [ ]:
for item in exposureList:
    expId = item[0]
    exp = butler.get('raw',  exposure=expId, detector=55, instrument="LSSTCam")
    mData = exp.getMetadata()
    print(expId, mData['CCOBFLUX'])


In [ ]:
repo = '/repo/ir2'
butler = daf_butler.Butler(repo, collections=["LSSTCam/raw/all","LSSTCam/calib","LSSTCam/calib/unbounded"])

expId=3023061800092
exp = butler.get('raw',  exposure=expId, detector=55, instrument="LSSTCam")
mData = exp.getMetadata()
for key in mData.keys():
    print(key, mData[key])

In [ ]:
exp = butler.get('postISRCCD',  exposure=expId, detector=55, instrument="LSSTCam")

In [ ]:
repo = '/repo/ir2'
operator = 'lsstccs'
acq_run = '13372'
weekly = 'w_2023_24'

butler = daf_butler.Butler(repo)
collections = butler.registry.queryCollections(f"u/{operator}/eo_*_{acq_run}_{weekly}",
                                               collectionTypes=daf_butler.CollectionType.CHAINED)
for item in collections:
    print(item)

In [ ]:
# Several eo_pipe pipelines produce per-amp measurements for all of the CCDs in the current Camera object.
# For LSSTCam, this would include all 205 CCDs in the focal plane; for TS8, this would be the 9 CCDs in 
# the raft installed at that test stand (currently RTM-004).
#
# The `eo_pipe.get_amp_data` function will retrieve these per-amp measurements and return them as a 
# three-level dictionary, keyed by measurement quantity, CCD (labeled by R**_S for LSSTCam), and 
# amplifier data (labeled by channel name, e.g., 'C01').
#
# The '[ps]cti' quantities are measured from combined flats, so will have different values depending on the
# filter used, since combined flats a created separately for each filter combination.
amp_data = eo_pipe.get_amp_data(repo, collections)
for quantity in amp_data.keys():
    print(quantity)

In [ ]:
# These dicts of amplifier data in a format that can be used with the focal plane plotting functions in eo_pipe:
plt.figure(figsize=(9, 9))
ax = plt.gca()
eo_pipe.plotting.plot_focal_plane(ax, amp_data['ptc_gain'], z_range=(1.0, 2.0));